In [12]:
import cv2
import os
import math
import dlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
from scipy.spatial import distance

from shapely.geometry import Polygon

#### Specifying detector and predictor

In [16]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/lewsz/OneDrive/Desktop/master_thesis/shape_predictor_68_face_landmarks.dat")

#### Text specification

In [17]:
font = cv2.FONT_HERSHEY_SIMPLEX
org = (236, 135)
fontScale = 0.2
color = (255, 255, 255)
thickness = 1

#### drawing landmarks

In [4]:
for k in range(1, 81):
    
    image_path = ("C:/Users/lewsz/OneDrive/Desktop/master_thesis/face_landmarks/faces/high_quality_dataset/hot/image_" +
                       str(k) + ".jpg")
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    rects = detector(image, 1)

    for (i, rect) in enumerate(rects):

        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)

        index = 0
        for (x, y) in shape:
            cv2.circle(image, (x, y), 1, (0, 255, 0), 3)
            cv2.putText(image, str(index), (x, y), font, 
                       fontScale, color, thickness, cv2.LINE_AA)
            index += 1

    
    saved_file_name = "faces_with_landmarks/picture_" + str(k) + ".png"
    plt.imsave(saved_file_name, image)

In [13]:
def facial_landmark_processor(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    rects = detector(image, 1)

    for (i, rect) in enumerate(rects):

        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)
    shape = shape.tolist()
    return shape

In [121]:
shape = facial_landmark_processor("faces/high_quality_dataset/not_hot/image_72.jpg")

#### Creating a function that will calculate our parameters

In [175]:
def parameter_processor(shape):
    
    #creating a dictionary, that will be added later to a dataframe
    ratio_dictionary =  {}
    
    #parameters that will be used for counting ratios and creating target variables
    
    right_eye_length = distance.euclidean(shape[42], shape[45])
    left_eye_length = distance.euclidean(shape[36], shape[39])
    
    right_eye_points = [shape[42], shape[43], shape[44], shape[45], shape[46], shape[47]]
    right_eye_points = [tuple(x) for x in right_eye_points]
    right_eye_area = Area(right_eye_points)
    
    left_eye_points = [shape[36], shape[37], shape[38], shape[39], shape[40], shape[41]]
    left_eye_points = [tuple(x) for x in left_eye_points]
    left_eye_area = Area(left_eye_points)
    
    
    eye_distance = distance.euclidean(shape[39], shape[42])
    eye_outer_distance = distance.euclidean(shape[36], shape[45])
    
    right_horizontal_eyebrow_width = distance.euclidean(shape[22], shape[26])
    left_horizontal_eyebrow_width = distance.euclidean(shape[36], shape[39])
    
    right_eyebrow_length = (distance.euclidean(shape[22], shape[23]) + 
                            distance.euclidean(shape[23], shape[24]) +
                            distance.euclidean(shape[24], shape[25]) +
                            distance.euclidean(shape[25], shape[26]))
                
    left_eyebrow_length = (distance.euclidean(shape[17], shape[18]) + 
                            distance.euclidean(shape[18], shape[19]) +
                            distance.euclidean(shape[19], shape[20]) +
                            distance.euclidean(shape[20], shape[21]))      
    
    right_vision_points = [shape[22], shape[23], shape[24], shape[25], shape[26], 
                           shape[45], shape[46], shape[47], shape[42]]
    right_vision_points = [tuple(x) for x in right_vision_points]
    right_vision_area = Area(right_vision_points)
    
    left_vision_points = [shape[17], shape[18], shape[19], shape[20], shape[21], 
                           shape[39], shape[40], shape[41], shape[39]]
    left_vision_points = [tuple(x) for x in left_vision_points]
    left_vision_area = Area(left_vision_points)
    
    
    
    nose_width = distance.euclidean(shape[31], shape[35])
    nose_length = distance.euclidean(shape[27], shape[33])
    
    nose_points = [shape[31], shape[32], shape[33], shape[34], shape[35], shape[27]]
    nose_points = [tuple(x) for x in nose_points]
    nose_area = Area(nose_points)
    
    mouth_length = distance.euclidean(shape[48], shape[54])
    mouth_width = distance.euclidean(shape[51], shape[57])
    
    upper_lip_points = [shape[48], shape[49], shape[50], shape[51], shape[52], shape[54],
                       shape[64], shape[63], shape[65], shape[66], shape[60]]
    upper_lip_points = [tuple(x) for x in upper_lip_points]
    upper_lip_area = Area(upper_lip_points)
    
    lower_lip_points = [shape[48], shape[59], shape[58], shape[57], shape[56], shape[55],
                       shape[54], shape[64], shape[65], shape[66], shape[67], shape[60]]
    lower_lip_points = [tuple(x) for x in lower_lip_points]
    lower_lip_area = Area(lower_lip_points)
    
    
    left_cupid_lip_width = distance.euclidean(shape[50], shape[58])
    right_cupid_lip_width = distance.euclidean(shape[52], shape[56])
    
    chin_to_mouth_distance = distance.euclidean(shape[8], shape[57])
    mouth_to_nose_distance = distance.euclidean(shape[51], shape[33])
    nose_to_chin_distance = distance.euclidean(shape[8], shape[33])
    
    left_eye_corner_to_left_lip_corner_length = distance.euclidean(shape[36], shape[49])
    right_eye_corner_to_left_lip_corner_length = distance.euclidean(shape[45], shape[54])
    
    left_lip_corner_to_chin_distance = distance.euclidean(shape[48], shape[8])
    right_lip_corner_to_chin_distance = distance.euclidean(shape[54], shape[8])
    
    left_bottom_face_to_left_lip_corner_length = distance.euclidean(shape[4], shape[48])
    right_bottom_face_to_right_lip_corner_length = distance.euclidean(shape[12], shape[54])
    
    face_length = abs(shape[19][1] - shape[8][1])
    
    face_width_at_bottom = distance.euclidean(shape[4], shape[12])
    face_width_at_top = distance.euclidean(shape[1], shape[15])
    
    face_points = [shape[19], shape[18], shape[17],shape[0], shape[1], shape[2], shape[3], 
                   shape[4], shape[5], shape[6],shape[7], shape[8], shape[9], shape[0], 
                   shape[11], shape[12],shape[13], shape[14], shape[15], shape[16], 
                   shape[26], shape[25], shape[24]]
    face_points = [tuple(x) for x in face_points]
    face_area = Area(face_points)
    
    inter_nose_mouth_area_points = [shape[31], shape[32], shape[33], shape[34], shape[35], shape[54], shape[48]]
    inter_nose_mouth_area_points = [tuple(x) for x in inter_nose_mouth_area_points]
    inter_nose_mouth_area = Area(face_points)
    
    under_mouth_points = [shape[48], shape[54], shape[10], shape[9], shape[8], shape[7], shape[6]]
    under_mouth_points = [tuple(x) for x in under_mouth_points]
    under_mouth_area = Area(under_mouth_points)
    
    bottom_face_points = [shape[5], shape[6], shape[7], shape[8], shape[9], shape[10], shape[11]]
    bottom_face_points = [tuple(x) for x in bottom_face_points]
    bottom_face_area = Area(bottom_face_points)
    
    lower_face_points = [shape[5], shape[4], shape[33], shape[13], shape[12], shape[11]]
    lower_face_points = [tuple(x) for x in lower_face_points]
    lower_face_area = Area(lower_face_points)
    
    upper_face_points = [shape[13], shape[14], shape[15], shape[1], shape[2], shape[3]]
    upper_face_points = [tuple(x) for x in upper_face_points]
    upper_face_area = Area(upper_face_points)
    
    top_face_points = [shape[1], shape[0], shape[17], shape[18], shape[19], shape[24],
                       shape[25], shape[26], shape[16], shape[15], shape[60]]
    top_face_points = [tuple(x) for x in top_face_points]
    top_face_area = Area(top_face_points)
    
    inner_triangle_points = [shape[0], shape[16], shape[8]]
    inner_triangle_points = [tuple(x) for x in inner_triangle_points]
    inner_triangle_area = Area(inner_triangle_points)
    
    right_outer_face_points = [shape[8], shape[9], shape[10], shape[11], shape[12], shape[13],
                       shape[14], shape[15], shape[16]]
    right_outer_face_points = [tuple(x) for x in right_outer_face_points]
    right_outer_face_area = Area(right_outer_face_points)
    
    left_outer_face_points = [shape[0], shape[1], shape[2], shape[3], shape[4], shape[5],
                       shape[6], shape[7], shape[8]]
    left_outer_face_points = [tuple(x) for x in left_outer_face_points]
    left_outer_face_area = Area(left_outer_face_points)
    
    features_points = [shape[17], shape[18], shape[19], shape[24], shape[25], shape[26],
                    shape[54], shape[55], shape[56], shape[57], shape[58], shape[59], shape[48]]
    features_points = [tuple(x) for x in features_points]
    features_area = Area(features_points)
    
    
    # CALCULATING RATIOS
    
    #symmetry
    eye_area_symmetry = (right_eye_area / left_eye_area)
    ratio_dictionary['s1'] = eye_area_symmetry
    
    eye_length_symmetry = (right_eye_length/left_eye_length)
    ratio_dictionary['s2'] = eye_length_symmetry
    
    
    
    #area ratios
    eyes_to_lips = ((right_eye_area + left_eye_area) / (upper_lip_area + lower_lip_area))
    ratio_dictionary['a1'] = eyes_to_lips
    
    eyes_to_nose = ((right_eye_area + left_eye_area) / nose_area)
    ratio_dictionary['a2'] = eyes_to_nose
    
    eyes_to_face = ((right_eye_area + left_eye_area) / face_area)
    ratio_dictionary['a3'] = eyes_to_face
    
    eyes_to_top_face = ((right_eye_area + left_eye_area) / top_face_area)
    ratio_dictionary['a4'] = eyes_to_top_face
    
    lips_to_nose = ((upper_lip_area + lower_lip_area) / nose_area)
    ratio_dictionary['a5'] = lips_to_nose
    
    lips_to_face = ((upper_lip_area + lower_lip_area) / face_area)
    ratio_dictionary['a6'] = lips_to_face
    
    lips_to_lower_face = ((upper_lip_area + lower_lip_area) / lower_face_area)
    ratio_dictionary['a7'] = lips_to_lower_face
    
    nose_to_face = (nose_area / face_area)
    ratio_dictionary['a8'] = nose_to_face
    
    eyes_lips_nose_to_face = ((right_eye_area + left_eye_area + upper_lip_area + lower_lip_area + nose_area) / face_area)
    ratio_dictionary['a9'] = eyes_lips_nose_to_face
    
    eyes_lips_nose_to_inner_face = ((right_eye_area + left_eye_area + upper_lip_area + lower_lip_area + nose_area) 
                                    / inner_triangle_area)
    ratio_dictionary['a10'] = eyes_lips_nose_to_inner_face
    
    features_to_triangle = (features_area / inner_triangle_area)
    ratio_dictionary['a11'] = features_to_triangle
    
    features_to_face = features_area / face_area
    ratio_dictionary['a12'] = features_to_face
    
    features_to_outer = features_area / (left_outer_face_area + right_outer_face_area)
    ratio_dictionary['a13'] = features_to_outer
    
    inner_to_outer = (inner_triangle_area / (left_outer_face_area + right_outer_face_area))
    ratio_dictionary['a14'] = inner_to_outer
    
    inner_to_face = inner_triangle_area / face_area
    ratio_dictionary['a15'] = inner_to_face
    
    outer_to_face = (left_outer_face_area + right_outer_face_area) / face_area
    ratio_dictionary['a16'] = outer_to_face
    
    top_face_to_face = top_face_area / face_area
    ratio_dictionary['a17'] = top_face_to_face
    
    upper_face_to_face = upper_face_area / face_area
    ratio_dictionary['a18'] = upper_face_to_face
    
    lower_face_to_face = lower_face_area / face_area
    ratio_dictionary['a19'] = lower_face_to_face
    
    bottom_face_to_face = bottom_face_area / face_area
    ratio_dictionary['a20'] = bottom_face_to_face
    
    upper_to_bottom = upper_face_area / bottom_face_area
    ratio_dictionary['a21'] = upper_to_bottom
    
    vision_to_face = (right_vision_area + left_vision_area) / face_area
    ratio_dictionary['a22'] = vision_to_face
    
    #distances
    
    face_length_to_width_top = (face_length / face_width_at_top)
    ratio_dictionary['d1'] = face_length_to_width_top
    
    face_length_to_width_bottom = (face_length / face_width_at_bottom)
    ratio_dictionary['d2'] = face_length_to_width_bottom
    
    face_top_to_bottom_width = face_width_at_top/face_width_at_bottom
    ratio_dictionary['d3'] = face_top_to_bottom_width
    
    eye_distance_to_face_width = (eye_distance /  face_width_at_top)
    ratio_dictionary['d4'] = eye_distance_to_face_width
    
    eyebrows_to_face_width = ((right_eyebrow_length + left_eyebrow_length) / face_width_at_top)
    ratio_dictionary['d5'] = eyebrows_to_face_width
    
    mouth_to_eye_distance = (mouth_length / eye_distance)
    ratio_dictionary['d6'] = mouth_to_eye_distance
    
    mouth_to_eye_spread = (mouth_length / eye_outer_distance)
    ratio_dictionary['d7'] = mouth_to_eye_spread
    
    mouth_to_nose_width = (mouth_length / nose_width)
    ratio_dictionary['d8'] = mouth_to_nose_width
    
    nose_length_to_face_length = (nose_length/face_length)
    ratio_dictionary['d9'] = nose_length_to_face_length
    
    bottom_distance_to_face = (chin_to_mouth_distance / face_length)
    ratio_dictionary['d10'] = bottom_distance_to_face
    
    nose_to_mouth_to_face =   (mouth_to_nose_distance / face_length)
    ratio_dictionary['d11'] = nose_to_mouth_to_face
       
    eye_distance_to_nose_width = (eye_distance / nose_width)
    ratio_dictionary['d12'] = eye_distance_to_nose_width
    
    nose_length_to_nose_to_chin_distance = nose_length / nose_to_chin_distance
    ratio_dictionary['d13'] = nose_length_to_nose_to_chin_distance
    
    nose_width_to_nose_mouth_distance = nose_width / mouth_to_nose_distance
    ratio_dictionary['d14'] = nose_width_to_nose_mouth_distance
    
    
    
    #distances and areas
    
    eyebrows_to_eyes = ((right_eyebrow_length + left_eyebrow_length) / (right_eye_area + left_eye_area))
    ratio_dictionary['da1'] = eyebrows_to_eyes
    
    vision_to_face_width = ((right_vision_area + left_vision_area) / face_width_at_top)
    ratio_dictionary['da2'] = vision_to_face_width
    
    
                                                 
    return ratio_dictionary

In [83]:
def Area(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

In [185]:
def data_creator():
    
    colnames = ['s1', 's2', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9',
                'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19',
                'a20', 'a21', 'a22', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9',
                'd10', 'd11', 'd12', 'd13', 'd14', 'da1', 'da2']
    final_dataset = pd.DataFrame(columns = colnames)
    
    for k in range(1, 81):
        image_path = ("C:/Users/lewsz/OneDrive/Desktop/master_thesis/face_landmarks/faces/high_quality_dataset/hot/image_" +
                           str(k) + ".jpg") 
        shape = facial_landmark_processor(image_path)
        ratios = parameter_processor(shape)
    
        final_dataset = final_dataset.append(ratios, ignore_index = True)
    
    return final_dataset

In [186]:
dataset = data_creator()

In [178]:
parameter_processor(shape)

{'s1': 1.020451921491011,
 's2': 1.0063637362977578,
 'a1': 0.5307625649913345,
 'a2': 0.5311307665626084,
 'a3': 0.027466675635377077,
 'a4': 0.03796118971918016,
 'a5': 1.0006937218175511,
 'a6': 0.0517494590746533,
 'a7': 0.2159431137724551,
 'a8': 0.0517135842330071,
 'a9': 0.13092971894303748,
 'a10': 0.19872516956333824,
 'a11': 1.0599129467096375,
 'a12': 0.6983217300642384,
 'a13': 1.9391086704770444,
 'a14': 1.829498051228411,
 'a15': 0.6588481933653965,
 'a16': 0.3601251135102411,
 'a17': 0.7235462280967275,
 'a18': 0.39837666341550915,
 'a19': 0.2396439421966614,
 'a20': 0.11864482785681454,
 'a21': 3.3577246527449685,
 'a22': 0.13728853462482762,
 'd1': 0.9464671210269572,
 'd2': 1.2040345906221042,
 'd3': 1.272135675791543,
 'd4': 0.2725691532286113,
 'd5': 0.7437967771975378,
 'd6': 1.2149717786429721,
 'd7': 0.5103981454668925,
 'd8': 1.8857569904325617,
 'd9': 0.3851752192913231,
 'd10': 0.1910171003338714,
 'd11': 0.09895570293244131,
 'd12': 1.5520994179295293,
 'd13'

In [188]:
dependent_variable = np.ones(80)
dataset['dependent_variable'] = dependent_variable

In [190]:
dataset.to_csv('group_1.csv')